In [1]:
import torch
import numpy as np
from utils.generate_csbm import *
from utils.train import *
from utils.utils import *
from models.models import *
from models.setup import *

In [6]:
import torch.nn.functional as F

In [2]:
csbm = cSBM(N=200, p=100, d=5, mu=0.5, l=2)
csbm.generate_node_parameters()
csbm.phi

0.8886112459769363

In [3]:
csbm.generate_node_data(method="GC", n_local=120)

In [4]:
A_tilde = calculate_Atilde(csbm.A, 10, 0.9)

In [7]:
server = set_up_GC(csbm.Xs, csbm.ys, MLP(100,64,2,False), A_tilde, n_train=60, n_val=30)
tl, ta, vl, va = train_GC(server, num_communication=500, 
               batch_size=40, learning_rate=0.1, I=10,
               gradient=True, noise=False,
               Print=True)

Communication: 1 Average train loss: 0.69344 Average train accuracy: 0.443 Average val loss: 0.69348 Average val accuracy: 0.434
Communication: 51 Average train loss: 0.69303 Average train accuracy: 0.510 Average val loss: 0.69299 Average val accuracy: 0.516
Communication: 101 Average train loss: 0.69265 Average train accuracy: 0.510 Average val loss: 0.69256 Average val accuracy: 0.517
Communication: 151 Average train loss: 0.69225 Average train accuracy: 0.511 Average val loss: 0.69213 Average val accuracy: 0.517
Communication: 201 Average train loss: 0.69176 Average train accuracy: 0.511 Average val loss: 0.69162 Average val accuracy: 0.517
Communication: 251 Average train loss: 0.69111 Average train accuracy: 0.511 Average val loss: 0.69098 Average val accuracy: 0.518
Communication: 301 Average train loss: 0.69025 Average train accuracy: 0.512 Average val loss: 0.69013 Average val accuracy: 0.519
Communication: 351 Average train loss: 0.68906 Average train accuracy: 0.514 Average v